In [1]:
import pandas as pd
import glob
import imageio
import numpy
from astropy.time import Time
from h5py import File
import numpy as np
import os
#from tqdm import tqdm
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import plotly.graph_objs as go

In [2]:
def load_hdf5_data(file_name: str):
    """
    Reads a datafile in HDF5 format using the config file set earlier.
    """
    file: File = File(file_name)
    time = file['Time']
    freqs = file['Frequency']
    data = file['Data']

    # sorted data to avoid issue later
    idx_sorted = np.argsort(time)
    time, data = time[idx_sorted], data[idx_sorted]

    return time, freqs, data

In [3]:
def polar_ephemeris_load_xyz(file, time_view_start, time_view_end, coordinate_system = "GSM"):
    df = pd.read_csv(file, delimiter = " ", header = None, skiprows=4,
                    names=["Date", "Time", "x_geo", "y_geo", "z_geo", "GEO_LAT", "GEO_LONG", "x_gm", "y_gm", "z_gm", "GM_LAT", "GM_LONG", "x_gse", "y_gse", "z_gse", "GSE_LAT", "GSE_LONG", "LT_gsm", "x_gsm", "y_gsm", "z_gsm", "GSM_LAT", "GSM_LONG", "Distance_R_E"])
    df["Date"] = "19"+df["Date"] 
    df.index = pd.to_datetime(df["Date"]+"T"+df["Time"])
    df.drop(["Date", "Time"], axis = 1, inplace = True)

    #Restricting flux and ephemeris data to be within the user defined start/end times
    
    df = df[(df.index > time_view_start) & (df.index < time_view_end)]

    date = numpy.array(Time(df.index).unix)
    x = numpy.array(df["x_"+coordinate_system.lower()], dtype=float)
    y = numpy.array(df["y_"+coordinate_system.lower()], dtype=float)
    z = numpy.array(df["z_"+coordinate_system.lower()], dtype=float)

    return(date,x,y,z)

In [4]:
def ms(x, y, z, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)

(x_pns_surface, y_pns_surface, z_pns_suraface) = ms(0,0,0,1)

In [5]:
def RotZ(t):
    return np.array([[cos(t), -sin(t), 0],
                     [sin(t), cos(t), 0],
                     [0, 0, 1]])

In [6]:
def make_gif(directory_path: str = './', files_name: str ='filename*.png', output_path: str ='output.gif', fps: int =10, delete_pngs: bool =False) -> None:
    # directory_path: Set the path to the directory containing the PNG files
    # output_path: Set the output filename and path

    # Use glob to get a list of all the PNG files in the directory
    png_files = sorted(glob.glob(directory_path+files_name))

    # Use imageio to create the GIF
    with imageio.get_writer(output_path, mode='I', fps=fps) as writer:
        for file in png_files:
            with open(file,'rb') as f:
                image = imageio.imread(f)
                writer.append_data(image)
            if delete_pngs: 
                os.remove(file)
    return


In [7]:
def plot_trajectory_3D(fig: go.Figure, x_all_orbit: np.ndarray, y_all_orbit: np.ndarray, z_all_orbit: np.ndarray, colours_all_orbit: np.ndarray, coordinate_system_name: str = "Unknown", filename: str = '3D_plot_trajectory', extension: str = "png", save_html: bool = False, save_image: bool = False, show_plot: bool = True, movie: bool = False, images_number: int = 10) -> None:
   
    # x_all_orbit, y_all_orbit, z_all_orbit are 1d numpy arrayes containgin the (x,y,z) coordinates
    # colours_all_orbit is a 1d numpy array containing flux values (same size than x, y, and z arrays)
    # filename needs to be without extension
    # extension can be png or pdf
    
    
    
    cmax_all_orbit =  np.percentile(colours_all_orbit, 90)
    cmin_all_orbit = np.percentile(colours_all_orbit, 35)

    fig.add_trace(
        go.Scatter3d(
            x=x_all_orbit,
            y=y_all_orbit,
            z=z_all_orbit,
            mode='markers',
            marker=dict(
                size=2,
                color=colours_all_orbit,
                cmax=cmax_all_orbit,
                cmin=cmin_all_orbit,
                colorbar=dict(
                    title=dict(
                        text="AKR Intensity (log V<sup>2</sup>/m<sup>2</sup>/Hz)",
                        side="right"
                    )
                ),
                colorscale="Viridis",
                opacity=0.8,
            ),
            showlegend=False
        )
    )

    fig.add_trace(
        go.Surface(
            x=x_pns_surface, y=y_pns_surface, z=z_pns_suraface,
            opacity=1,
            colorscale="greys",
            lightposition=dict(x=-10000, y=0, z=0),
            showlegend=False,
            showscale=False
        )
    )

    fig.update_layout(
        font=dict(size=12),
        scene=dict(
            xaxis=dict(range=[-5, 5]),
            yaxis=dict(range=[-5, 5]),
            zaxis=dict(range=[-2, 10]),
            xaxis_title=f"X ({coordinate_system_name})",
            yaxis_title=f"Y ({coordinate_system_name})",
            zaxis_title=f"Z ({coordinate_system_name})",
            camera_eye=dict(x=2.5, y=0, z=0.1)
        ),
        margin=dict(t=1, r=1, b=1, l=1)
    )

    if movie:
        angle_circle = np.linspace(0, 2*np.pi, images_number)
        radius_circle = 2.5
        x_circle = radius_circle * np.cos(angle_circle)
        y_circle = radius_circle * np.sin(angle_circle)

        for k in tqdm(range(len(x_circle))):
            fig.update_scenes(camera_eye=dict(x=x_circle[k], y=y_circle[k], z=0.1))
            fig.write_image(f"{filename}_{k+1:03d}.{extension}", scale=3)
    else:
        if save_html:
            fig.write_html(filename+".html") 
        if save_image:
            fig.write_image(f"{filename}.{extension}", scale = 3)
        if show_plot:
            fig.show()
           


In [8]:
dir_save_name = "/Users/clouis/Documents/Data/POLAR/data"
data_path_ephemeris = glob.glob("/Users/clouis/Documents/Data/POLAR/data/polar_ephemeris_geo_gm_gse_gsm.txt")[0]
time_view_start_all_orbit = '1996-03-25'
time_view_end_all_orbit ='1996-05-03'

coordinate_system_table = ["GM", "GSM"]#["GSE", "GEO", "GSM", "GM"]
type_feature = "all_features"
path_masked_data = f"/Users/clouis/Documents/Data/POLAR/data/polar_{type_feature}_mask.hdf5"

# load masked data
(time_data, freqs_data, masked_data) = load_hdf5_data(path_masked_data)


for coordinate_system in tqdm(coordinate_system_table):
    # load ephemeris
    date_ephem_all_orbit, x_all_orbit, y_all_orbit, z_all_orbit = polar_ephemeris_load_xyz(data_path_ephemeris, time_view_start_all_orbit, time_view_end_all_orbit, coordinate_system=coordinate_system)

    # Interpolate the ephemeris data so it lines up with the flux data
    idx_mask = (time_data > date_ephem_all_orbit[0]) & (time_data < date_ephem_all_orbit[-1])
    masked_data_all_orbit = masked_data[idx_mask]
    time_data_all_orbit = time_data[idx_mask]

    x_all_orbit = np.interp(time_data_all_orbit, date_ephem_all_orbit, x_all_orbit)
    y_all_orbit = np.interp(time_data_all_orbit, date_ephem_all_orbit, y_all_orbit)
    z_all_orbit = np.interp(time_data_all_orbit, date_ephem_all_orbit, z_all_orbit)
    date_ephem_all_orbit = np.interp(time_data_all_orbit, date_ephem_all_orbit, date_ephem_all_orbit)

    # Normalizing @ 1 AU
    RE = 6371.0
    AU = 149597870.700  # 1 astronomical unit
    distance = np.sqrt(x_all_orbit**2 + y_all_orbit**2 + z_all_orbit**2)
    distance_factor = (distance * RE / AU) ** 2
    masked_data_all_orbit *= distance_factor[:, None]

    colours_all_orbit = np.log10(masked_data_all_orbit.sum(axis=1))

    filename = f"polar_trajectory_{coordinate_system}_{type_feature}"

    fig = go.Figure()
    images_number = 360
    make_movie = True
    extension = "png"
    print(f"### Plotting {images_number} figure(s) ... this may take some time... ###")

    plot_trajectory_3D(fig, x_all_orbit, y_all_orbit, z_all_orbit, colours_all_orbit, coordinate_system_name=coordinate_system, filename=filename, extension=extension, save_html=False, save_image=False, show_plot=False, movie=make_movie, images_number = images_number)
    if make_movie:
        print(f"### Making gif from {images_number} {extension} figure(s) ... this may take some time... ###")
        make_gif(directory_path = './', files_name=f'{filename}*.png', output_path=f'{filename}.gif', fps=10, delete_pngs=True)

  0%|          | 0/2 [00:00<?, ?it/s]

### Plotting 360 figure(s) ... this may take some time... ###


<ipython-input-8-aa9a43f1c510>:35: RuntimeWarning: divide by zero encountered in log10
  colours_all_orbit = np.log10(masked_data_all_orbit.sum(axis=1))


  0%|          | 0/360 [00:00<?, ?it/s]

### Making gif from 360 png figure(s) ... this may take some time... ###
### Plotting 360 figure(s) ... this may take some time... ###


<ipython-input-8-aa9a43f1c510>:35: RuntimeWarning:

divide by zero encountered in log10



  0%|          | 0/360 [00:00<?, ?it/s]

### Making gif from 360 png figure(s) ... this may take some time... ###


In [9]:
make_gif(directory_path = './', files_name=f'{filename}*.png', output_path=f'{filename}.gif', fps=10, delete_pngs=True)